In [49]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [41]:
df=pd.read_csv("spy_intraday.csv")
# df.rename(columns={"Time":"time","Open":"open","High":"high","Low":"low","Last":"close","Volume":"volume","Change":"change"}, inplace=True)
df.rename(columns={"Last":"Close"}, inplace=True)
len(df)

147047

In [42]:
# Viewing non-trading days
df.loc[df["Volume"]==0]

,Time,Open,High,Low,Close,Change,Volume
87668,12/24/19 15:30,321.23,321.23,321.23,321.23,0.00,0
87669,12/24/19 13:10,321.23,321.23,321.23,321.23,-0.01,0
94121,11/29/19 15:30,314.31,314.31,314.31,314.31,0.00,0
94122,11/29/19 13:10,314.31,314.31,314.31,314.31,-0.02,0
134495,7/3/19 15:30,298.80,298.80,298.80,298.80,0.00,0
134496,7/3/19 13:10,298.80,298.80,298.80,298.80,0.00,0


In [43]:
# Dropping non-trading days from dataset
df.drop(index=holidays.index, inplace=True)
df

,Time,Open,High,Low,Close,Change,Volume
0,11/13/20 15:59,358.350,358.42,357.9300,358.1000,-0.2500,2920759
1,11/13/20 15:58,358.205,358.35,358.1400,358.3500,0.1450,718921
2,11/13/20 15:57,357.825,358.21,357.8118,358.2050,0.3850,479111
3,11/13/20 15:56,357.860,357.95,357.8000,357.8200,-0.0589,309294
4,11/13/20 15:55,357.920,357.96,357.8100,357.8789,-0.0411,492569
...,...,...,...,...,...,...,...
147042,5/17/19 11:55,287.585,287.62,287.4800,287.5000,-0.0800,88436
147043,5/17/19 11:54,287.500,287.58,287.4300,287.5800,0.0700,60199
147044,5/17/19 11:53,287.430,287.57,287.4050,287.5100,0.0800,116036
147045,5/17/19 11:52,287.375,287.44,287.2700,287.4300,0.0500,63474


In [44]:
old_close=1
old_volume=1
close_list=[]
for index, row in df.iterrows():
    close=row["Close"]
    volume=row["Volume"]
    
    # Calculating differences in price and volume
    close_diff=close-old_close
    volume_diff=volume-old_volume
    
    # Creating binary outcome of closing price change compared to previous day
    if close_diff>0:
    # Closed higher
        close_change=1
    else:
    # Closed lower or no change
        close_change=0
        
    # Calculating percent changes
    price_swing=row["High"]-row["Low"]
    price_percent_change=((close-old_close)/old_close)*100
    volume_change_percent=((volume-old_volume)/old_volume)*100
    
    # Calculating moving averages
    if len(close_list)==10:
        close_list.pop(0)
        
    close_list.append(close)
    avg_10_day=sum(close_list)/len(close_list)
    
    if len(close_list)>=5:
        avg_5_day=sum(close_list[:5])/5
    else:
        avg_5_day=sum(close_list[:5])/len(close_list[:5])
        
    if len(close_list)>=2:
        avg_2_day=sum(close_list[:2])/2
    else:
        avg_2_day=sum(close_list[:2])/len(close_list[:2])
    
    # Adding values to DataFrame
    df.loc[index, "price_change_$"]=close_diff
    df.loc[index, "price_change_%"]=price_percent_change
    df.loc[index, "volume_change"]=volume_diff
    df.loc[index, "volume_change_%"]=volume_change_percent
    df.loc[index, "2_day_avg_$"]=avg_2_day
    df.loc[index, "5_day_avg_$"]=avg_5_day
    df.loc[index, "10_day_avg_$"]=avg_10_day
    df.loc[index, "daily_volatility_$"]=price_swing
    df.loc[index, "price_change_binary"]=close_change
    
    # Replacing closing information
    old_close=close
    old_volume=volume

In [46]:
# The price_change and volume_change columns in the first row are not scaled properly
# This is because there was no previous day's values to compare to
df.head()

,Time,Open,High,Low,Close,Change,Volume,price_change_$,price_change_%,volume_change,volume_change_%,2_day_avg_$,5_day_avg_$,10_day_avg_$,daily_volatility_$,price_change_binary
0,11/13/20 15:59,358.350,358.42,357.9300,358.1000,-0.2500,2920759,357.1000,35710.000000,2920758.0,2.920758e+08,358.100,358.100000,358.100000,0.4900,1.0
1,11/13/20 15:58,358.205,358.35,358.1400,358.3500,0.1450,718921,0.2500,0.069813,-2201838.0,-7.538582e+01,358.225,358.225000,358.225000,0.2100,1.0
2,11/13/20 15:57,357.825,358.21,357.8118,358.2050,0.3850,479111,-0.1450,-0.040463,-239810.0,-3.335693e+01,358.225,358.218333,358.218333,0.3982,0.0
3,11/13/20 15:56,357.860,357.95,357.8000,357.8200,-0.0589,309294,-0.3850,-0.107480,-169817.0,-3.544419e+01,358.225,358.118750,358.118750,0.1500,0.0
4,11/13/20 15:55,357.920,357.96,357.8100,357.8789,-0.0411,492569,0.0589,0.016461,183275.0,5.925592e+01,358.225,358.070780,358.070780,0.1500,1.0


In [54]:
# Creating new DataFrame that excludes the first row of data
df_final=df.iloc[1:,:]
df_final.rename(columns={"Time":"time","Open":"open","High":"high","Low":"low","Close":"close","Volume":"volume","Change":"open_change_$"}, inplace=True)
df_final

,time,open,high,low,close,open_change_$,volume,price_change_$,price_change_%,volume_change,volume_change_%,2_day_avg_$,5_day_avg_$,10_day_avg_$,daily_volatility_$,price_change_binary
1,11/13/20 15:58,358.205,358.35,358.1400,358.3500,0.1450,718921,0.2500,0.069813,-2201838.0,-75.385816,358.2250,358.225000,358.225000,0.2100,1.0
2,11/13/20 15:57,357.825,358.21,357.8118,358.2050,0.3850,479111,-0.1450,-0.040463,-239810.0,-33.356934,358.2250,358.218333,358.218333,0.3982,0.0
3,11/13/20 15:56,357.860,357.95,357.8000,357.8200,-0.0589,309294,-0.3850,-0.107480,-169817.0,-35.444187,358.2250,358.118750,358.118750,0.1500,0.0
4,11/13/20 15:55,357.920,357.96,357.8100,357.8789,-0.0411,492569,0.0589,0.016461,183275.0,59.255918,358.2250,358.070780,358.070780,0.1500,1.0
5,11/13/20 15:54,358.075,358.15,357.9100,357.9200,-0.1550,389637,0.0411,0.011484,-102932.0,-20.896971,358.2250,358.070780,358.045650,0.2400,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147042,5/17/19 11:55,287.585,287.62,287.4800,287.5000,-0.0800,88436,0.0400,0.013915,-42318.0,-32.364593,287.3275,287.395000,287.448500,0.1400,1.0
147043,5/17/19 11:54,287.500,287.58,287.4300,287.5800,0.0700,60199,0.0800,0.027826,-28237.0,-31.929305,287.4000,287.448000,287.480000,0.1500,1.0
147044,5/17/19 11:53,287.430,287.57,287.4050,287.5100,0.0800,116036,-0.0700,-0.024341,55837.0,92.754032,287.4800,287.484000,287.492000,0.1650,0.0
147045,5/17/19 11:52,287.375,287.44,287.2700,287.4300,0.0500,63474,-0.0800,-0.027825,-52562.0,-45.298011,287.4550,287.492000,287.494000,0.1700,0.0


In [55]:
# Save DataFrame as CSV file
df_final.to_csv("intraday_transformed.csv")